In [73]:
import subprocess

In [74]:
# Exercice 2

# Question 1
# n_j * n_e * n_e variables (nombre de 3-tuples possibles entre trois ensembles finis et dénombrables)

# Question 2
def codage(n_e, j, x, y):
    return j*(n_e**2) + x*n_e + y + 1 # cf énoncé

def decodage(k, n_e): # meme logique que la conversion secondes -> hh:mm:ss
    j = (k - 1) // n_e**2
    x = ((k - 1) % n_e**2) // n_e
    y = (k - 1) % n_e
    return j,x,y

In [75]:
# Exercice 3
# Question 1

def au_moins_une(liste): # clause au format DIMACS
    return (" ".join([str(_) for _ in liste])) + " 0\n" # il suffit de valuer à vrai une des variables propositionnelles

def au_plus_une(liste): # clause au format DIMACS
    dimacs = ""
    for i in range(len(liste)):
        for j in range(len(liste)):
            if j > i:
                clause = []
                clause.append(-liste[i])
                clause.append(-liste[j])
                dimacs += (" ".join([str(_) for _ in clause])) + " 0\n"
    
    return dimacs

QUESTION 2

1) La contrainte C1 peut s'écrire :

$\forall x \in \{0,\ldots,n_e -1\}, \forall j \in \{0, \ldots, n_j - 1\}, \sum_{y \in \{0, \ldots, n_e - 1 \}} m_{j,x,y} + m_{j,y,x} \leq 1$

In [76]:
# Question 2
# 2) 

def encoderC1(n_e,n_j):
    c1 = ""
    for j in range(n_j):
        for x in range(n_e):
            adv_equipe_jour = [] # liste contenant les matchs potentiels d'une equipe pour un jour donné
            for y in range(n_e):
                if x != y:
                    k1 = codage(n_e,j,x,y) # domicile
                    k2 = codage(n_e,j,y,x) # ext
                    adv_equipe_jour.append(k1)
                    adv_equipe_jour.append(k2)
            c1 += au_plus_une(adv_equipe_jour) # au plus une equipe y peut etre jouée par x pour un jour j

    return c1


# 3)
# print("Nombre de contraintes :", len(encoderC1(3,4).splitlines()))
# print(encoderC1(3,4))

4. La contrainte C2 peut s'écrire :

$\forall x \in \{0,\ldots,n_e -1\}, \forall y \in \{0, \ldots, n_e - 1\}, \sum_{j \in \{0, \ldots, n_j - 1 \}} m_{j,x,y} = 1 \text{ et } m_{j,y,x} = 1$

In [77]:
# 5)

def encoderC2(n_e,n_j):
    c2 = ""
    for x in range(n_e):
        for y in range(n_e):
            if x!=y: # une equipe ne peut pas jouer contre elle-meme
                x_vs_y = []
                for j in range(n_j):
                    k = codage(n_e,j,x,y)
                    x_vs_y.append(k)
                c2 += au_plus_une(x_vs_y)
                c2 += au_moins_une(x_vs_y) # exactement une rencontre entre x (domicile) et y (exterieur), quel que soit le jour (donc dans le championnat entier)
    return c2

# 6)
# print("Nombre de contraintes :", len(encoderC2(3,4).splitlines()))
# print(encoderC2(3,4))


# 7)
def encoder(n_e, n_j):
    return encoderC1(n_e, n_j) + encoderC2(n_e, n_j)

In [78]:
# Question 3

def nb_variables(n_e, n_j):
    return n_j * n_e * n_e # d'après 1)

def ecrire_cnf(n_e, n_j, filepath):
    f = open(filepath, "w")
    s = encoder(n_e, n_j)
    nb_cont = len(('' + s).splitlines()) # ''+s pour ne pas faire pointer sur s l'operation
    f.write("p cnf " + str(nb_variables(n_e, n_j)) + " " + str(nb_cont) + "\n") # premiere ligne format DIMACS
    f.write(s) # ecrit automatique a la suite (coup de bol)
    f.close()

Le résultat renvoyé par Glucose est que l'ensemble de clauses est insatisfiable. Normal, il n'y a pas assez de journées pour accueillir toutes les rencontres. Il faudrait en ajouter 2 à 4 pour que 3 équipes puissent s'affronter à des journées différentes à domicile et à l'extérieur (voir tout en bas pour le raisonnement).

In [79]:
# Question 4

def decoder(filepath, n_j, n_e, equipes=None):
    b = False # pour renvoyer si le decodage a donné quelque chose ou pas
    planning = []
    for j in range(n_j):
        planning.append([])
        
    if equipes != None:
        noms = open(equipes, 'r').read().splitlines()
    with open(filepath, 'r') as f:
        lines = f.read().splitlines()[-2:]
        if lines[-1].split(" ")[-1] == "UNSATISFIABLE":
            return b
        else:
            line = lines[-1].split(" ")[1:-1] # on enleve le v du debut et le 0 de la fin
            for item in line:
                k = int(item)
                if k >= 0:
                    b = True
                    j,x,y = decodage(k, n_e)
                    if equipes != None:
                        x = noms[x]
                        y = noms[y]
                    planning[j].append((x,y))
            
            print("Planning :")
            for i in range(len(planning)):
                print("Journée", i+1, ":")
                for (x,y) in planning[i]:
                    print(x, "vs", y)
    
    return b

In [81]:
# Question 5
'''
ATTENTION : BIEN ETRE DANS LE REPERTOIRE OÙ GLUCOSE_STATIC SE TROUVE
'''

# n_e = int(input("Entrer le nombre d'équipes : "))
# n_j = int(input("Entrer le nombre de journées de championnat : "))
teams = "equipes.txt"

ecrire_cnf(n_e, n_j, "championnat.cnf")
resultat = subprocess.run(["./glucose_static", "-model", "championnat.cnf"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
res = resultat.stdout # on recupere la sortie de l'appel à glucose pour la resolution SAT
f = open("resultat.txt", "w")
f.write(res)
f.close()
#decoder("resultat.txt", n_j, n_e, equipes=teams)

In [82]:
# Exercice 4
import time

def optimiser_nj(ne_min=3, ne_max=10):
    
    nj_opt = [0] * (ne_max - ne_min + 1) # liste de 0 du nombre d'équipes à tester
    
    for n_e in range(ne_min, ne_max+1):
        n_j = 1
        t_start = time.time()
        t_fin = t_start
        while t_fin - t_start < 10: # 10 secondes
            ecrire_cnf(n_e, n_j, "championnat.cnf")
            resultat = subprocess.run(["./glucose_static", "-model", "championnat.cnf"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            res = resultat.stdout # on recupere la sortie de l'appel à glucose pour la resolution SAT
            f = open("resultat.txt", "w")
            f.write(res)
            f.close()
            if decoder("resultat.txt", n_j, n_e) != False:
                nj_opt[n_e - ne_min] = n_j
                break
            n_j += 1
            t_fin = time.time()
            
    return nj_opt

def borne_min_nj(n_e):
    return (n_e-1)*n_e # chaque équipe rencontre 2 fois une autre, sans pouvoir jouer contre elle-meme, n_e equipes jouent contre n-1 autres

In [83]:
# print(optimiser_nj())